# Impact of Energy Price Changes

In this notebook, we analyse how shifts in energy prices influence consumption across sectors and countries using official SDMX data.

In [1]:
from utils import utils
import json
import plotly.express as px
from math import e
import pandas as pd


with open('utils/sub_codelists.json', 'r') as f:
    sub_codelists = json.load(f)

For this analysis we need to use three datasets:
- Energy balances for the consumption
- Energy prices
- ECB's Main Aggregates National Accounts for the gdp

Eurostat only publishes energy prices for electricity and gas, separating between households and not-households. Therefore, we will focus our analysis there. In order to use the data, several arrangements are required.

In [2]:
nrg_bal_constraints = {
    'freq': 'A',
    'nrg_bal': ['FC_E','FC_OTH_HH_E'],
    'siec': ['E7000','G3000'],
    'unit': 'GWH',
    'geo': sub_codelists['geo_eu_countries'],
}

#We are interested in: https://data.ecb.europa.eu/data/datasets/MNA/MNA.A.N.AT.W2.S1.S1.B.B1GQ._Z._Z._Z.EUR.V.N
mna_constraints = {
    "FREQ": "A",
    "ADJUSTMENT": "N",
    "REF_AREA":  sub_codelists['geo_eu_countries'],
    "COUNTERPART_AREA": "W2",
    "REF_SECTOR": "S1",
    "COUNTERPART_SECTOR": "S1",
    "ACCOUNTING_ENTRY": "B",
    "STO": "B1GQ",
    "INSTR_ASSET": "_Z",
    "ACTIVITY": "_Z",
    "EXPENDITURE": "_Z",
    "UNIT_MEASURE": "EUR",
    "PRICES": "V",
    "TRANSFORMATION": "N"
}

#nrg_prices do not have values for CY, FI, MT. Therefore, we need to drop them from the geo list
geo_list = [country for country in sub_codelists['geo_eu_countries'] if country not in ['CY', 'FI', 'MT']]

nrg_prices_gas_constraints = {
    'freq': 'S',
    "product": '',
    'nrg_cons': '',
    'unit':'KWH',
    'tax': ['I_TAX', 'X_TAX', 'X_VAT'],
    'currency':'EUR',
    'geo':geo_list
}

nrg_pices_electricity_constraints = {
    'freq': 'S',
    'product': '',
    'nrg_cons': '',
    'unit': 'KWH',
    'tax': ['I_TAX', 'X_TAX', 'X_VAT'],
    'currency':'EUR',
    'geo':geo_list
}

gdp = utils.get_dataset_with_selection('MNA', mna_constraints, agency='ecb')
nrg_bal = utils.get_dataset_with_selection('nrg_bal_s', nrg_bal_constraints, agency='estat')
gas_prices_hh = utils.get_dataset_with_selection('nrg_pc_202', nrg_prices_gas_constraints, agency='estat')
gas_prices_nhh = utils.get_dataset_with_selection('nrg_pc_203', nrg_prices_gas_constraints, agency='estat')
electricity_prices_hh = utils.get_dataset_with_selection('nrg_pc_204', nrg_pices_electricity_constraints, agency='estat')
electricity_prices_nhh = utils.get_dataset_with_selection('nrg_pc_205', nrg_pices_electricity_constraints, agency='estat')



# Arranging the dgp dataframe
gdp_df = gdp.data[["REF_AREA", "TIME_PERIOD", "OBS_VALUE", "UNIT_MULT"]]
gdp_df.loc[:, "OBS_VALUE"] = gdp_df["OBS_VALUE"].astype(float) * (10 ** gdp_df["UNIT_MULT"].astype(int))
gdp_df = gdp_df[["REF_AREA", "TIME_PERIOD", "OBS_VALUE"]]

# Arranging the energy balances dataset, which we sued to get the consumption.
nrg_bal_df = nrg_bal.data

nrg_bal_df_hh = nrg_bal_df[nrg_bal_df["nrg_bal"] == "FC_OTH_HH_E"]
nrg_bal_df_hh = nrg_bal_df_hh.drop(columns=["nrg_bal"])
nrg_bal_df_hh.rename(columns={"OBS_VALUE": "HH_VALUE"}, inplace=True)

nrg_bal_df_total = nrg_bal_df[nrg_bal_df["nrg_bal"] == "FC_E"]
nrg_bal_df_total = nrg_bal_df_total.drop(columns=["nrg_bal"])
nrg_bal_df_total.rename(columns={"OBS_VALUE": "TOTAL_VALUE"}, inplace=True)

nrg_bal_df_nhh = pd.merge(nrg_bal_df_hh, nrg_bal_df_total, on=['siec', 'geo', 'TIME_PERIOD'])
nrg_bal_df_nhh["NHH_VALUE"] = nrg_bal_df_nhh["TOTAL_VALUE"] - nrg_bal_df_nhh["HH_VALUE"]
nrg_bal_df_nhh = nrg_bal_df_nhh[['siec', 'geo', 'TIME_PERIOD', "NHH_VALUE"]]

nrg_bal_df_hh.rename(columns={"HH_VALUE": "OBS_VALUE"}, inplace=True)
nrg_bal_df_hh["nrg_bal"] = "FC_OTH_HH_E"
nrg_bal_df_nhh.rename(columns={"NHH_VALUE": "OBS_VALUE"}, inplace=True)
nrg_bal_df_nhh["nrg_bal"] = "FC_NHH_E"

nrg_bal_df = pd.concat([nrg_bal_df_hh, nrg_bal_df_nhh], ignore_index=True)
nrg_bal_df.rename(columns={"OBS_VALUE": "GWh consumption"}, inplace=True)

# Arranging the energy prices dataframe
gas_prices_hh_df = gas_prices_hh.data
gas_prices_hh_df['nrg_bal'] = 'FC_OTH_HH_E'
gas_prices_hh_df['siec'] = 'G3000'

gas_prices_nhh_df = gas_prices_nhh.data
gas_prices_nhh_df['nrg_bal'] = 'FC_NHH_E'
gas_prices_nhh_df['siec'] = 'G3000'

electricity_prices_hh_df = electricity_prices_hh.data
electricity_prices_hh_df['nrg_bal'] = 'FC_OTH_HH_E'
electricity_prices_hh_df['siec'] = 'E7000'

electricity_prices_nhh_df = electricity_prices_nhh.data
electricity_prices_nhh_df['nrg_bal'] = 'FC_NHH_E'
electricity_prices_nhh_df['siec'] = 'E7000'

energy_prices_df = pd.concat([gas_prices_hh_df, gas_prices_nhh_df, electricity_prices_hh_df, electricity_prices_nhh_df], ignore_index=True)

energy_prices_df = energy_prices_df.drop(columns=['OBS_FLAG', 'CONF_STATUS'])
energy_prices_df['TIME_PERIOD'] = energy_prices_df['TIME_PERIOD'].str[0:4].astype(int)
energy_prices_df = energy_prices_df.groupby(['geo', 'TIME_PERIOD', 'nrg_bal', 'siec', 'tax'])['OBS_VALUE'].mean().reset_index()
energy_prices_df = pd.pivot_table(energy_prices_df, values='OBS_VALUE', index=['geo', 'TIME_PERIOD', 'nrg_bal', 'siec'], columns=['tax'])
energy_prices_df.reset_index(inplace=True)
#We calculate the applied taxes to energy
energy_prices_df['% tax'] = round((energy_prices_df['I_TAX'] - energy_prices_df['X_TAX']) / energy_prices_df['I_TAX'] * 100,2)
energy_prices_df.rename(columns={
    "I_TAX": "EUR per Kwh Including tax",
    "X_TAX": "EUR per Kwh Excluding tax",
    "X_VAT": "EUR per Kwh Excluding VAT and other recoverable taxes",
    "% tax": "Percentage of taxes applied to energy"}, inplace=True)

Let's first check the gas price by country

In [21]:
electricity_prices_hh_df = energy_prices_df[
    (energy_prices_df['nrg_bal'] == 'FC_NHH_E') &
    (energy_prices_df['siec'] == 'G3000') 
]

fig = px.line(electricity_prices_hh_df, x='TIME_PERIOD', y='EUR per Kwh Excluding tax', color='geo')
fig.show()

This chart displays energy prices in euros per kilowatt-hour (EUR per kWh) excluding taxes across different European countries from 2007 to 2024.
Key observations:

- Energy prices remained relatively stable between 0.02-0.05 EUR/kWh for most countries from 2007 to 2019
- A dramatic price spike occurred in 2021-2022, with peaks reaching as high as 0.14 EUR/kWh for some countries
- All countries experienced this significant price surge, though with varying magnitudes
- By 2023-2024, prices began declining but remained higher than pre-2020 levels
- The synchronization of this price spike across all countries suggests a common external factor, most likely the energy crisis following Russia's invasion of Ukraine and subsequent disruptions to European energy markets
- There's considerable convergence in price movements across countries, though with persistent differences in absolute levels

This visualization powerfully illustrates the European energy crisis of 2021-2022 and its impact on consumer energy prices. The unprecedented nature of this price shock is evident when comparing the relatively stable pre-2020 period with the dramatic spike and subsequent partial recovery in the following years.

Let's now check the policy reactions to these changes by analysing the taxes to gas:

In [14]:
electricity_prices_hh_df = energy_prices_df[
    (energy_prices_df['nrg_bal'] == 'FC_NHH_E') &
    (energy_prices_df['siec'] == 'G3000') 
]

fig = px.line(electricity_prices_hh_df, x='TIME_PERIOD', y='Percentage of taxes applied to energy', color='geo')
fig.show()

Key observations:

- Denmark (DK, light blue line at top) consistently maintains the highest energy tax rates, ranging between 50-65% until 2020
- Most countries maintain tax rates in the 15-30% range through most of the period
- There's a notable disruption around 2021-2022, with dramatic drops in tax percentages for several countries, including some falling to negative values (particularly visible for Greece and Denmark)
- By 2024, most tax rates appear to be recovering toward their pre-2021 levels

This chart directly relates to the previous visualization of energy prices excluding taxes. The dramatic drop in tax percentages around 2021-2022 coincides precisely with the spike in energy prices shown in the previous chart. This strongly suggests that many European governments responded to the energy crisis by implementing tax reductions or subsidies to mitigate the impact of skyrocketing energy prices on consumers.
The negative tax rates observed for some countries indicate that governments were providing subsidies that exceeded the normal tax amount – essentially paying part of the base energy price to protect consumers from the full impact of the price surge. As energy prices began normalizing in 2023-2024, these emergency tax cuts and subsidies are gradually being phased out, returning toward pre-crisis levels.

Finally, we are going to calculate the total cost of gas and electricity consumption as a percentage of the GDP, by country. The total cost is calculated by multiplying the consumption by the price of energy.

In [17]:
prices_and_comsumption = pd.merge(energy_prices_df, nrg_bal_df, on=['geo', 'TIME_PERIOD', 'siec', 'nrg_bal'])
prices_and_comsumption['total_consumption'] = prices_and_comsumption['GWh consumption'] * prices_and_comsumption['EUR per Kwh Excluding tax'] * 10**6

prices_and_consumption_gas = prices_and_comsumption[prices_and_comsumption['siec'] == 'G3000']
cost_of_gas = prices_and_consumption_gas.groupby(['geo', 'TIME_PERIOD'])['total_consumption'].sum().reset_index()

gdp_df.rename(columns={"OBS_VALUE": "GDP", "REF_AREA": "geo"}, inplace=True)

cost_of_gas_over_gdp = pd.merge(cost_of_gas, gdp_df, on=['geo', 'TIME_PERIOD'])
cost_of_gas_over_gdp['Cost of gas consumption over GDP'] = cost_of_gas_over_gdp['total_consumption'] / cost_of_gas_over_gdp['GDP']

fig = px.line(cost_of_gas_over_gdp, x='TIME_PERIOD', y='Cost of gas consumption over GDP', color='geo')
fig.show()

Key observations:

- Slovakia (SK, green line at top initially) and Hungary (HU, blue line) show the highest gas costs relative to GDP in the earlier years, reaching around 0.025-0.028 (2.5-2.8% of GDP)
- Most countries show a general declining trend from 2012 to 2020, suggesting improving energy efficiency or reduced reliance on gas relative to economic output
- A significant spike occurs for all countries in 2021-2022, coinciding with the European energy crisis shown in previous charts
- By 2023, the ratios begin declining again but remain above pre-crisis levels

This chart complements the previous visualizations by showing the economic burden of energy costs. The spike in 2021-2022 aligns perfectly with the price surge seen in the energy prices chart, but this visualization emphasizes the macroeconomic impact of the crisis.
The data reveals that despite government interventions through tax reductions and subsidies (as shown in the previous chart), the energy crisis still resulted in gas costs consuming a significantly larger share of GDP across Europe. This suggests that while policy measures helped mitigate the impact on consumers, the overall economic burden of higher energy prices was substantial.
The varying heights of lines also highlight differences in gas dependency and energy efficiency across European economies, with some eastern European countries showing persistently higher gas-to-GDP ratios throughout the period.

And for the electricity consumption:

In [18]:
prices_and_consumption_electricity = prices_and_comsumption[prices_and_comsumption['siec'] == 'E7000']
cost_of_electricity = prices_and_consumption_electricity.groupby(['geo', 'TIME_PERIOD'])['total_consumption'].sum().reset_index()

cost_of_electricity_over_gdp = pd.merge(cost_of_electricity, gdp_df, on=['geo', 'TIME_PERIOD'])
cost_of_electricity_over_gdp['Cost of electricity over GDP'] = cost_of_electricity_over_gdp['total_consumption'] / cost_of_electricity_over_gdp['GDP']

fig = px.line(cost_of_electricity_over_gdp, x='TIME_PERIOD', y='Cost of electricity over GDP', color='geo')
fig.show()


Key observations:

- Bulgaria (BG, green line) consistently shows the highest electricity costs relative to GDP for most of the period, maintaining levels around 0.04-0.05 (4-5% of GDP)
- There's a dramatic spike in Bulgaria's ratio in 2022, reaching approximately 0.072 (7.2% of GDP)
- Most countries show relatively stable electricity-to-GDP ratios in the 0.01-0.03 range from 2007 to 2020
- Similar to the previous gas consumption chart, there's a significant spike for all countries in 2021-2022 during the European energy crisis
- By 2023, the ratios begin declining again but remain above pre-crisis levels for most countries
- Denmark (DK, light blue near bottom) consistently maintains among the lowest electricity costs relative to GDP despite having some of the highest tax rates as shown in previous charts

This visualization complements the previous charts by showing the economic impact of electricity costs specifically. When viewed alongside the gas consumption chart, it reveals:

- The energy crisis affected both gas and electricity costs relative to GDP
- The magnitude of the electricity cost spike in 2022 was particularly severe for Bulgaria
- The overall pattern of stable costs followed by crisis-driven spikes and partial recovery is consistent across both energy types
- Countries show varying degrees of electricity intensity in their economies, suggesting differences in efficiency, industrial structure, and electrification levels